# Solved Example: 1D SLAM — Weighted Least Squares

**Question:** 

A robot navigates in 1D environment and closes loop by returning to starting point. Our measurements or observed values are given by:

$$\begin{array}{ccc}\hline \text { Quantity } & \text { Ground Truth } & \text { Observed / Measured } \\\hline \mathbf{u}_{0} & 1.0 & 1.1 \\\mathbf{u}_{1} & 1.0 & 1.0 \\\mathbf{u}_{2} & 1.0 & 1.1 \\\mathbf{u}_{3} & -3.0 & -2.7 \\\mathbf{u}_{0,4} & 0.0 & 0.0 \\\hline\end{array}$$

$\mathbf{u_0} \rightarrow \mathbf{u_3}$ is obtained through odometry control inputs from an odometer/IMU.
$\mathbf{u_{0,4}}$ is obtained from a loop closure method, say Bag of Visual Words.
<img src="./misc/pose_graph_1.png" width=500 height=600>

Where the big dots $x_{i}$ are the poses at timestamps $t_{i}$, the blue dots are the transformations between two subsequent poses and the green dot is the transformation between two loop-closure detections (need not be subsequent)

Note that in a SLAM setting, we wouldn't have the ground truth. It's been mentioned above so that we can compare and know if we are improving after we apply our algorithm. 

Above, there are 4 odometry constraints of the form $\left\|f\left(\mathbf{x}_{i}, \mathbf{u}_{i}\right)-\mathbf{x}_{i+1}\right\|_{\mathbf{\Sigma}_{i}}^{2}$ and one loop closure of the form $\left\|f\left(\mathbf{x}_{0}, \mathbf{u}_{0,4}\right)-\mathbf{x}_{4}\right\|_{\mathbf{\Lambda}_{0,4}}^{2}$. [Recollect here](https://www.notion.so/Pose-graph-SLAM-Foundations-1D-Solved-Example-9044a74a66724f46a6dabf490369a324). Standard deviations for both are initialized as $0.1$. ($\boldsymbol{\sigma}_{i}=0.01 \text { and } \boldsymbol{\lambda}_{0,4}=0.01$)

Now the question is to get a better estimate of the robot states through optimization.

**Answer:** 

Next robot state is given by (A simple motion model given by $f$) :

$$\mathbf{x}_{i+1} = f\left(\mathbf{x}_{i}, \mathbf{u}_{i}\right) =  \mathbf{x}_i + \mathbf{u}_i$$

Robot states according to odometry alone:

$$\begin{array}{ccc}\hline \text { Quantity } & \text { Ground Truth } & \text { According to Odometry } \\\hline \mathrm{x}_{0} & 0.0 & 0.0 \\\mathrm{x}_{1} & 1.0 & 1.1 \\\mathrm{x}_{2} & 2.0 & 2.1 \\\mathrm{x}_{3} & 3.0 & 3.2 \\\mathrm{x}_{4} & 0.0 & 0.5 \\\hline\end{array}$$

According to odometry, $\mathbf{x}_{4}=0.5,$ but according to the loop closure constraint $\mathbf{x}_{4}=\mathbf{x}_{0}=0.0$ (In the [below residual](https://www.notion.so/1e9fe184f77f4fac9c47f171a41a43ce),  $\mathbf{x}_{4}=0.5$ but  $\mathbf{x}_{0} + \mathbf{u}_{0,4} = 0$.)

Note in the above table that according to loop closure constraint, $\mathbf{x}_{4}=\mathbf{x}_{0}=0.0$. Using this "additional" measurement, we want to "correct" our robot state which has gone wrong because odometry drifted eventually (which happens in all practical scenarios).

Our objective function is

$$\mathbf{F}(\mathbf{x})=\mathbf{f}(\mathbf{x})^{\top} \mathbf{\Omega} \mathbf{f}(\mathbf{x})$$

*Recall:*

$$\underset{X}{argmin}\medspace \underset{\textit{Odometry Constraints}}{\underbrace{\sum_{i }\lVert{f(x_{i},u_{i})-x_{i+1}}\rVert_{\sum_{i}}^2}}  + \underset{\textit{Loop Closure Constraints}}{\underbrace{\sum_{ij}\lVert{f(x_{i},u_{ij})-x_{j}}\rVert_{\Lambda_{ij}}^2}} $$

$$\left\|\mathbf{x}_{i}-\mathbf{y}_{i}\right\|_{\mathbf{\Sigma}_{i}}^{2}=\left(\mathbf{x}_{i}-\mathbf{y}_{i}\right)^{\top} \mathbf{\Sigma}_{i}^{-1}\left(\mathbf{x}_{i}-\mathbf{y}_{i}\right)$$

*So,*

$$\mathbf{f}(\mathbf{x})=\left(\begin{array}{c}f\left(\mathbf{x}_{0}, \mathbf{u}_{0}\right)-\mathbf{x}_{1} \\f\left(\mathbf{x}_{1}, \mathbf{u}_{1}\right)-\mathbf{x}_{2} \\f\left(\mathbf{x}_{2}, \mathbf{u}_{2}\right)-\mathbf{x}_{3} \\f\left(\mathbf{x}_{3}, \mathbf{u}_{3}\right)-\mathbf{x}_{4} \\f\left(\mathbf{x}_{0}, \mathbf{u}_{0,4}\right)-\mathbf{x}_{4} \\\mathbf{x}_{0}-\mathbf{0}\end{array}\right)=\left(\begin{array}{c}\mathbf{x}_{0}+\mathbf{u}_{0}-\mathbf{x}_{1} \\\mathbf{x}_{1}+\mathbf{u}_{1}-\mathbf{x}_{2} \\\mathbf{x}_{2}+\mathbf{u}_{2}-\mathbf{x}_{3} \\\mathbf{x}_{3}+\mathbf{u}_{3}-\mathbf{x}_{4} \\\mathbf{x}_{0}+\mathbf{u}_{0,4}-\mathbf{x}_{4} \\\mathbf{x}_{0}-0\end{array}\right)$$

The last line is a "prior" constraint that anchors the first pose $\mathbf{x}_0$ at the origin with a very less covariance 0.001. 

$$\mathbf{f(x_{O})} = (0 \quad 0  \quad 0 \quad 0 \quad -0.5 \quad 0)^T$$

$\mathbf{f(x_{O})}$ is the initialization of $\mathbf{f(x)}$

The information matrix encodes the uncertainty of each edge. The "bigger" $\Omega$ is, the more the edge matters in the optimization or the more confident we are about the pose. As of now, we have more confidence on the pose $\mathbf{x_0}$. You can think of it as the "inverse" effect of variance values. 

The information matrix is then given by:

$$\mathbf{\Omega}=\left(\begin{array}{cccccc}\mathbf{\Sigma}_{0} & \boldsymbol{0} & \boldsymbol{0} & \boldsymbol{0} & \boldsymbol{0} & \boldsymbol{0} \\\boldsymbol{0} & \mathbf{\Sigma}_{1} & \boldsymbol{0} & \boldsymbol{0} & \boldsymbol{0} & \boldsymbol{0} \\\boldsymbol{0} & \boldsymbol{0} & \mathbf{\Sigma}_{2} & \boldsymbol{0} & \boldsymbol{0} & \boldsymbol{0} \\\boldsymbol{0} & \boldsymbol{0} & \boldsymbol{0} & \mathbf{\Sigma}_{3} & \boldsymbol{0} & \boldsymbol{0} \\\boldsymbol{0} & \boldsymbol{0} & \boldsymbol{0} & \boldsymbol{0} & \mathbf{\Lambda}_{0,4} & \boldsymbol{0} \\\boldsymbol{0} & \boldsymbol{0} & \boldsymbol{0} & \boldsymbol{0} & \boldsymbol{0} & \mathbf{\Pi}\end{array}\right)^{-1}=\left(\begin{array}{cccccc}0.01 & 0 & 0 & 0 & 0 & 0 \\0 & 0.01 & 0 & 0 & 0 & 0 \\0 & 0 & 0.01 & 0 & 0 & 0 \\0 & 0 & 0 & 0.01 & 0 & 0 \\0 & 0 & 0 & 0 & 0.01 & 0 \\0 & 0 & 0 & 0 & 0 & 0.001\end{array}\right)^{-1}$$

$$=\left(\begin{array}{cccccc}100 & 0 & 0 & 0 & 0 & 0 \\0 & 100 & 0 & 0 & 0 & 0 \\0 & 0 & 100 & 0 & 0 & 0 \\0 & 0 & 0 & 100 & 0 & 0 \\0 & 0 & 0 & 0 & 100 & 0 \\0 & 0 & 0 & 0 & 0 & 1000\end{array}\right)$$

*Recall Gauss Newton:*

$$\mathbf{J}^{\top} \mathbf{\Omega} \mathbf{J} \Delta \mathbf{x}=-\mathbf{J}^{\top} \mathbf{\Omega}^{\top} \mathbf{f}(\mathbf{x})$$

$$\mathbf{f}(\mathbf{x})=\left(\begin{array}{c}f\left(\mathbf{x}_{0}, \mathbf{u}_{0}\right)-\mathbf{x}_{1} \\f\left(\mathbf{x}_{1}, \mathbf{u}_{1}\right)-\mathbf{x}_{2} \\f\left(\mathbf{x}_{2}, \mathbf{u}_{2}\right)-\mathbf{x}_{3} \\f\left(\mathbf{x}_{3}, \mathbf{u}_{3}\right)-\mathbf{x}_{4} \\f\left(\mathbf{x}_{0}, \mathbf{u}_{0,4}\right)-\mathbf{x}_{4} \\\mathbf{x}_{0}-\mathbf{0}\end{array}\right)=\left(\begin{array}{c}\mathbf{x}_{0}+\mathbf{u}_{0}-\mathbf{x}_{1} \\\mathbf{x}_{1}+\mathbf{u}_{1}-\mathbf{x}_{2} \\\mathbf{x}_{2}+\mathbf{u}_{2}-\mathbf{x}_{3} \\\mathbf{x}_{3}+\mathbf{u}_{3}-\mathbf{x}_{4} \\\mathbf{x}_{0}+\mathbf{u}_{0,4}-\mathbf{x}_{4} \\\mathbf{x}_{0}-0\end{array}\right)$$

$$\mathbf{J}=\frac{\partial \mathbf{f}}{\partial \mathbf{x}}=\left(\frac{\partial \mathbf{f}}{\partial \mathbf{x}_{0}} \frac{\partial \mathbf{f}}{\partial \mathbf{x}_{1}} \cdots \frac{\partial \mathbf{f}}{\partial \mathbf{x}_{4}}\right)=\left(\begin{array}{ccccc}1 & -1 & 0 & 0 & 0 \\0 & 1 & -1 & 0 & 0 \\0 & 0 & 1 & -1 & 0 \\0 & 0 & 0 & 1 & -1 \\1 & 0 & 0 & 0 & -1 \\1 & 0 & 0 & 0 & 0\end{array}\right)$$

***Do you realize now why we typically have an initial estimate in SLAM/Vision problems? And hence how that helps in optimization? (You'll realize even more after Computer Vision is taught)***

$$\mathbf{f({x}}_{O})  = (0 \quad 0  \quad 0 \quad 0 \quad -0.5 \quad 0)^T$$

$$\begin{array}{ccccc} &  {\mathbf{H} \Delta \mathbf{x}=-\mathbf{b}} \\& & & \\\mathbf{H}=\mathbf{J}^{\top} \mathbf{\Omega} \mathbf{J}= & \left(\begin{array}{ccccc}1200 & -100 & 0 & 0 & -100 \\-100 & 200 & -100 & 0 & 0 \\0 & -100 & 200 & -100 & 0 \\0 & 0 & -100 & 300 & -100 \\-100 & 0 & 0 & -100 & 200\end{array}\right)\end{array} \\ \mathbf{b}=\mathbf{J}^{\top} \mathbf{\Omega}^{\top} \mathbf{f}\left(\mathbf{x}_{{O}}\right)=\left(\begin{array}{ccccc}
-50 & 0 & 0 & 0 & 50
\end{array}\right)^{\top}$$

$$\Delta \mathbf{x}=\left(\begin{array}{lllll}0 & -0.1 & -0.2 & -0.3 & -0.4\end{array}\right)^{\mathrm{T}}$$

$$\mathbf{x}_{I}=\mathbf{x}_{O}+\Delta \mathbf{x}=\left(\begin{array}{lllll}0 & 1.0 & 1.9 & 2.9 & 0.1\end{array}\right)^T$$

$$\begin{array}{ccc}\hline \text { Quantity } & \text { Ground Truth } & \text { According to Odometry } \\\hline \mathrm{x}_{0} & 0.0 & 0.0 \\\mathrm{x}_{1} & 1.0 & 1.1 \\\mathrm{x}_{2} & 2.0 & 2.1 \\\mathrm{x}_{3} & 3.0 & 3.2 \\\mathrm{x}_{4} & 0.0 & 0.5 \\\hline\end{array}$$

$$\begin{array}{ccc}\hline \text { Quantity } & \text { Ground Truth } & \text { Observed / Measured } \\\hline \mathbf{u}_{0} & 1.0 & 1.1 \\\mathbf{u}_{1} & 1.0 & 1.0 \\\mathbf{u}_{2} & 1.0 & 1.1 \\\mathbf{u}_{3} & -3.0 & -2.7 \\\mathbf{u}_{0,4} & 0.0 & 0.0 \\\hline\end{array}$$

# Sparsity in SLAM

<img src="./misc/SLAM_registration.png" width=500 height=600>

The sparsity can be appreciated directly from looking at the above pose graph. It is clear from the above figure that the graph is sparse, i.e., it is by no means a fully connected graph. The odometry chain linking the 100 unknown poses is a linear structure of 100 binary factors, instead of the possible $100^{2}$ (binary) factors. In addition, with 20 landmarks we could have up to 2000 likelihood factors linking each landmark to each pose: the true number is closer to 400. And finally, there are no factors between landmarks at all. This reflects that we have not been given any information about their relative position. **This structure is typical of most SLAM problems.**

Let us understand this in detail by revisiting our solved example.

1. The normal equation can be using:
    
    $$\begin{array}{ccccc} &  {\mathbf{H} \Delta \mathbf{x}=-\mathbf{b}} \\& & & \\\mathbf{H}=\mathbf{J}^{\top} \mathbf{\Omega} \mathbf{J}= & \left(\begin{array}{ccccc}1200 & -100 & 0 & 0 & -100 \\-100 & 200 & -100 & 0 & 0 \\0 & -100 & 200 & -100 & 0 \\0 & 0 & -100 & 300 & -100 \\-100 & 0 & 0 & -100 & 200\end{array}\right)\end{array} \\ \mathbf{b}=\mathbf{J}^{\top} \mathbf{\Omega}^{\top} \mathbf{f}\left(\mathbf{x}_{\mathbf{0}}\right)=\left(\begin{array}{ccccc}
    -50 & 0 & 0 & 0 & 50
    \end{array}\right)^{\top}$$
    
2. $\mathbf{H}$ is an adjacency matrix of the factor graph representation of the SLAM problem, and it has non-zero values only where node $i$ and node $j$ are connected.

<img src="./misc/posegraph_loopclosures.png" width=500 height=600>

3. Most of the nodes in the factor graph representation are connected to two nodes, except the loop closure nodes. 
4. The nonzero elements in the $\mathbf{J}$ and $\mathbf{H}$ are marked by dark blue. The diagonal entries correspond to odometry measurements while off-diagonal entries are from loop closure constraints.

<img src="./misc/posegraph_jacobian.png" width=500 height=600>